# Magic the gathering strategy chatbot based on RAG

![](https://logos-world.net/wp-content/uploads/2023/05/Magic-The-Gathering-Logo.png)


A "magic" bot to discuss strategies, cards, effects of the fantastic game Magic the Gathering

The system is based on RAG using the pre-trained model Mistral 7B and all-mpnet-base-v2 as embedding

An actual application will be available soon! Enjoy!

## Part 0: pip + libraries + drive

In [ ]:
!pip install langchain
!pip install langchain-community
!pip install sentence-transformers
!pip install faiss-cpu
!pip install transformers
!pip install bitsandbytes
!pip install torch
!pip install accelerate
!pip install huggingface-hub -q
!pip install -U langchain-huggingface
!pip install --upgrade transformers accelerate langchain langchain-huggingface


In [2]:
import pandas as pd
from pathlib import Path
from matplotlib import pyplot as plt
import matplotlib.colors as mcolors
from google.colab import drive

In [ ]:
#Drive
running_on_colab = True
if running_on_colab:
    drive_path = '/content/drive'
    drive.mount(drive_path)

## Part 1: data processing

In [4]:
# Get the origial cards dataset and remove the not usefull columns
file_path = '/content/drive/MyDrive/Magic/cards_final.csv'
magic_cards = pd.read_csv(file_path)
magic_df = pd.DataFrame(magic_cards)

In [5]:
from langchain.document_loaders import DataFrameLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
# dataframe loader
loader = DataFrameLoader(magic_df, page_content_column='page_content')
card_info = loader.load()

In [7]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=150,
    length_function=len
)

cards_documents = text_splitter.transform_documents(card_info)

## Part 2: embeddings

In [8]:
from langchain.embeddings import CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain_huggingface import HuggingFaceEmbeddings

In [ ]:
cache_path = LocalFileStore('./cache/')

embedding_model_id = 'sentence-transformers/all-mpnet-base-v2'

# creation of the embedding given the selected model
embedding_model_HF = HuggingFaceEmbeddings(model_name = embedding_model_id)
embedder = CacheBackedEmbeddings.from_bytes_store(embedding_model_HF, cache_path, namespace = embedding_model_id)
vector_store = FAISS.from_documents(cards_documents, embedder)

## LOGIN

In [ ]:
from huggingface_hub import notebook_login

# login to hugginfacehub, copy and paste the token inside the box
token = 'INSERT TOKEN'

notebook_login()

## Part 3: LLM

In [13]:
import transformers
import torch
from langchain.llms import HuggingFacePipeline

In [ ]:
# Hugging Face model name
model_id = 'mistralai/Mistral-7B-Instruct-v0.2'

# config the for 4-bit quantization
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_use_double_quant = True,
    bnb_4bit_compute_dtype = torch.bfloat16
)

# set the desired model
model_config = transformers.AutoConfig.from_pretrained(model_id)

# load the model for causal language modeling
model = transformers.AutoModelForCausalLM.from_pretrained(
        model_id,
        trust_remote_code = True,
        config = model_config,
        quantization_config = bnb_config,
        torch_dtype = torch.bfloat16,
        device_map = "auto",
    )

# evaluation
model.eval()

# load the pre-trained tokenizer associated with the selected model
tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)

In [ ]:
# pipeline for text generation using our model and the tokenizer
text_generation_pipeline = transformers.pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    temperature=0.1,
    do_sample="False",
    torch_dtype=torch.bfloat16,
    repetition_penalty=1.2,
    return_full_text=True,
    max_new_tokens=300,
)
llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

In [16]:
#from langchain_core.prompts.prompt import PromptTemplate
#from langchain.chains import RetrievalQA
#from langchain.callbacks import StdOutCallbackHandler
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import ChatPromptTemplate

In [27]:
retriever = vector_store.as_retriever(search_type='similarity', search_kwargs = {"k" : 10})

template = (
    """
    <s>[INST]
    You are an assisant for question-answering tasks about Magic the gathering card game.
    Use the following pieces of retrieved context to
    answer the question. Combine the chat history and
    follow up question into a standalone question, if
    the chat history is relevant to the question. If
    you don't know the answer, say you don't know.
    If the chat history is not relevant to the question,
    do not focus on it.
    </s>
    [INST]
    Chat History: {chat_history}
    Follow up question: {question}
    [/INST]
    """
)
prompt = ChatPromptTemplate.from_template(template)
chain = ConversationalRetrievalChain.from_llm(llm, retriever, prompt, return_source_documents=True)

## Part 4

In [28]:
chat_history = []

question = "What does Firebrand Archer card?"
result = chain({"question": question, "chat_history": chat_history})

print(result['answer'])


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Name: Firebrand Archer
Type: Creature — Human Archer ['Human', 'Archer']
Text: Whenever you cast a noncreature spell, Firebrand Archer deals 1 damage to each opponent.
Power: 2
Toughness: 1

Name: Firebrand Archer
Type: Creature — Human Archer ['Human', 'Archer']
Text: Whenever you cast a noncreature spell, Firebrand Archer deals 1 damage to each opponent.
Power: 2
Toughness: 1

Name: Firebrand Archer
Type: Creature — Human Archer ['Human', 'Archer']
Text: Whenever you cast a noncreature spell, Firebrand Archer deals 1 damage to each opponent.
Power: 2
Toughness: 1

Name: Flame Javelin
Type: Instant 
Text: ({2/R} can be paid with any two mana or with {R}. This card's mana value is 6.)
Flame Javelin deals 4 damage to any target.
Power: 
Toughness:

Name: Flame Javelin
Type: Instant 
Text: ({2/R} can be paid with any two mana 

In [29]:
question = "What card do you suggest for me to gain many life points??"
result = chain({"question": question, "chat_history": chat_history})

print(result['answer'])

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Name: Union of the Third Path
Type: Instant 
Text: Draw a card, then you gain life equal to the number of cards in your hand.
Power: 
Toughness:

Name: Live Fast
Type: Sorcery 
Text: You draw two cards, lose 2 life, and get {E}{E} (two energy counters).
Power: 
Toughness:

Name: Live Fast
Type: Sorcery 
Text: You draw two cards, lose 2 life, and get {E}{E} (two energy counters).
Power: 
Toughness:

Name: Life Burst
Type: Instant 
Text: Target player gains 4 life, then gains 4 life for each card named Life Burst in each graveyard.
Power: 
Toughness:

Name: Heartwarming Redemption
Type: Instant 
Text: Discard all the cards in your hand, then draw that many cards plus one. You gain life equal to the number of cards in your hand.
Power: 
Toughness:

Name: Gerrard's Wisdom
Type: Sorcery 
Text: You gain 2 life for each card in you

In [35]:
question = "Build a deck based on the ability Fly and Life-link"
result = chain({"question": question, "chat_history": chat_history})

print(result['answer'])

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Name: Windwright Mage
Type: Artifact Creature — Human Wizard ['Human', 'Wizard']
Text: Lifelink (Damage dealt by this creature also causes you to gain that much life.)
Windwright Mage has flying as long as an artifact card is in your graveyard.
Power: 2
Toughness: 2

Name: Sky Crier
Type: Creature — Bird Citizen ['Bird', 'Citizen']
Text: Flying, lifelink
{3}{W}: You and target opponent each draw a card.
Power: 1
Toughness: 1

Name: Heron of Hope
Type: Creature — Bird ['Bird']
Text: Flying
If you would gain life, you gain that much life plus 1 instead.
{1}{W}: Heron of Hope gains lifelink until end of turn.
Power: 2
Toughness: 3

Name: Heron of Hope
Type: Creature — Bird ['Bird']
Text: Flying
If you would gain life, you gain that much life plus 1 instead.
{1}{W}: Heron of Hope gains lifelink until end of turn.
Power: 2
Toughn